*Author: Pooya Bagheri*               <b>Under Construction</b>

# 1. Machine Learning (ML) Objective

From now on, our objective is to create statistical learning (or machine learning) models to estimate feeder power loss from voltages at a specific instant. 



# 2. Training and Test Data Sets

The common practice is to randomly split the available data to training and test sets. However, we know our data here is coming from simulations that were generated by random instant load profiles. So, we can go ahead and pick few days of simulation results as Training and another few days for the Test, and it will be automatically a random split. Additionally, the other advantage of keeping time-continuous data is allowing for better visualizations.

Hence, for all the tests of ML models in this project we choose the last 5 of the available 30 days data. For training the linear models here, we grab data of the first 10 days For now we are not loading the whole 25 left days' data to avoid memory limitation problem. Later, we will take advantage of total data through iterative data streaming.
 
Ok, let's create our training and test sets accordingly. Similar to previous [step](Step1-DataExploration.ipynb), we have to use SQL query and also pivoting for the voltage data. In order to avoid repetitive coding, we define a class called *MLinputData* for loading data for generic period of simulation days. (Please refer back to [previous step](Step1-DataExploration.ipynb) for more information, if the below procedure of SQL query and dataframe pivoting of is not clear to your)

Definition of this class is loaded below. Please note that for any future data loading we will rather directly import and use this class.

In [10]:
# %load LoadingDataClass.py
# @author: Pooya Bagheri
import os 
import pandas as pd
from sqlalchemy import create_engine
# a class is defined for loading data and covering the SQL and pivoting procedure:
class MLinputData:
    def __init__(self,FromDay,ToDay):
        RawVoltages=pd.read_sql_query('''
        select Instants.InstantID,Voltages.NodeID,Voltages.Vmag
        from Instants join Voltages on Instants.InstantID=Voltages.InstantID
        where Instants.Day>=%d and Instants.Day<=%d''' % (FromDay,ToDay),con=DB)
        Voltages=RawVoltages.pivot(index='InstantID', columns='NodeID', values='Vmag')
        self.x=Voltages.values
        Ploss=pd.read_sql_query('''
        select Losses.Ploss
        from Instants join Losses on Instants.InstantID=Losses.InstantID
        where Instants.Day>=%d and Instants.Day<=%d''', % (FromDay,ToDay),con=DB)
        self.y=Ploss.values

SyntaxError: invalid syntax (<ipython-input-10-c45f89f9cbd0>, line 18)

In [2]:
train=MLinputData(1,5) # Loading Training sets (first 5 days)
test=MLinputData(26,30) # Loading Test sets (last 5 of 30 days)

3
